In [1]:
import json
from flask import Flask
from flask import request
import word_segmentation as ws
import news_similarity as ns

import extract_future_class as exf
import future_label as fl


WordSegmentation = ws.WordSegmentation()
NewsSimilarity = ns.NewsSimilarity()
ExtractFuture = exf.ExtractFuture()
FutureLabel = fl.FutureLabel()

app = Flask(__name__)
Counter = 0

@app.route('/')
def hello_world():
    return 'Hello World!'

@app.route('/data/v1.0/getFutureLabels/', methods=['POST'])
def getFutureLabels():
    global Counter
#     print(request.form['title'])
#     print(request.form['content'])
    str_time = request.form['time']
    str_title = request.form['title']
    str_content = request.form['content']
    print("-------------")
    print("time:",str_time)
    print("title:",str_title)
    
    # 分词
    news_seg = WordSegmentation.word_segmentation(str_title + str_content)
    content_seg = WordSegmentation.word_segmentation(str_content)
    

    # 去重
    repeat = NewsSimilarity.news_similarity(content_seg)
    Counter += 1
    if Counter >= 100:
        NewsSimilarity.delete_1h_news()
        NewsSimilarity.tf_idf_model()
        Counter = 0


    # 提取期货关键词
    future_keywords = ExtractFuture.extract_future(news_list)
    
    # 预测期货分类
    label = FutureLabel.future_classification(news_list,str_future_list)
    
    dic_output = dict(repeat = repeat, plate = label)
    json_output = json.dumps(dic_output)
    return json_output
    
if __name__ == '__main__':
    app.run("0.0.0.0")


load 1h news success
tf-idf model has beens build successfully
